In [1]:
#Import Dependencies
import pandas as pd
import os
import time
import requests
import pymongo
from splinter import Browser
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
#path to driver
executable_path = {'executable_path':r'C:\Users\Jordan\OneDrive\Desktop\chromedriver'}
browser = Browser('chrome', **executable_path, headless = False)

### NASA Mars News

In [3]:
#Visit URL
url = 'https://redplanetscience.com/'
browser.visit(url)

In [4]:
#HTML Object
html = browser.html

#Parse with beatiful soup
soup = bs(html, 'html.parser')

In [5]:
#collect the latest News Title and Paragraph Text
news_title = soup.find_all('div', class_='content_title')[0].text
news_p = soup.find_all('div', class_='article_teaser_body')[0].text

#Display scraped data 
print(news_title)
print("-----------------------------------------------")
print(news_p)

NASA's Curiosity Mars Rover Takes a New Selfie Before Record Climb
-----------------------------------------------
Along with capturing an image before its steepest ascent ever, the robotic explorer filmed its "selfie stick," or robotic arm, in action.


### JPL Mars Space Images - Featured Image

In [6]:
#Visit URL
image_url = 'https://spaceimages-mars.com'
browser.visit(image_url)

In [7]:
#HTML Object
image_html = browser.html

#Parse with beautiful soup
image_soup =  bs(image_html, 'html.parser')

In [8]:
#Find image url full size
image_path = image_soup.find_all('img')[3]["src"]

#Find image link
featured_image_url = image_url + image_path
print(featured_image_url)

https://spaceimages-mars.comimage/mars/Proctor Crater Dunes 7.jpg


### Mars Facts

In [9]:
#Visit URL
facts_url = 'https://galaxyfacts-mars.com'
browser.visit(facts_url)

In [10]:
#Use Pandas to scrape the table containing facts about the planet
table = pd.read_html(facts_url)
table

[                         0                1                2
 0  Mars - Earth Comparison             Mars            Earth
 1                Diameter:         6,779 km        12,742 km
 2                    Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 3                   Moons:                2                1
 4       Distance from Sun:   227,943,824 km   149,598,262 km
 5          Length of Year:   687 Earth days      365.24 days
 6             Temperature:     -87 to -5 °C      -88 to 58°C,
                       0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:          2 ( Phobos & Deimos )
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC

In [12]:
#Create dataframe
facts_df = table[0]
facts_df

,0,1,2
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [14]:
mars_facts_df = facts_df.drop(labels=0, axis=0)
mars_facts_df

,0,1,2
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [15]:
mars_facts_df.columns = ['Description','Mars', 'Earth']
mars_facts_df

,Description,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [16]:
#convert the data to a HTML table string
mars_facts_html = mars_facts_df.to_html()
print(mars_facts_html)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Description</th>
      <th>Mars</th>
      <th>Earth</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>1</th>
      <td>Diameter:</td>
      <td>6,779 km</td>
      <td>12,742 km</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg</td>
      <td>5.97 × 10^24 kg</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Moons:</td>
      <td>2</td>
      <td>1</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Distance from Sun:</td>
      <td>227,943,824 km</td>
      <td>149,598,262 km</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Length of Year:</td>
      <td>687 Earth days</td>
      <td>365.24 days</td>
    </tr>
    <tr>
      <th>6</th>
      <td>Temperature:</td>
      <td>-87 to -5 °C</td>
      <td>-88 to 58°C</td>
    </tr>
  </tbody>
</table>


### Mars Hemispheres

In [17]:
#Visit URL
astro_url = 'https://marshemispheres.com/'
browser.visit(astro_url)

In [18]:
#HTML Object
hemisphere_html = browser.html

#Parse with beautiful soup
hemisphere_soup = bs(hemisphere_html, 'html.parser')

In [19]:
#Retrieve all items
hemispheres = hemisphere_soup.find_all('div', class_='item')

# #Create empty list
hemisphere_img_urls = []

# # #Main URL for loop
hemi_url = 'https://marshemispheres.com/'

for i in hemispheres:
    title = i.find('h3').text
    hemispheres_img = i.find('a', class_='itemLink product-item')['href']

# Visit full image website link
    browser.visit(hemi_url + hemispheres_img)

# HTML Object
    hemi_img_html = browser.html
    hemi_info = bs(hemi_img_html, 'html.parser')

# Create full image url
    img_url = hemi_url + hemi_info.find('img', class_='wide-image')['src']
    hemisphere_img_urls.append({'title' : title, 'img_url' : img_url})

print(hemisphere_img_urls)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]
